In [129]:
import random
import pandas as pd
import numpy as np
import numpy.linalg as la
import scipy.sparse as sp
import sklearn.preprocessing
from scipy.sparse.linalg import svds
from sklearn.metrics import mean_squared_error as MSE
import matplotlib.pyplot as plt
import itertools


df = pd.read_csv('user_song.csv')
test_df = pd.read_csv('test.csv')
train_users = df.iloc[:,0].unique().tolist()
train_songs = df.iloc[:,1].unique().tolist()
train_couples = list(zip(df.iloc[:,0], df.iloc[:,1]))

test_users = test_df.iloc[:,0].unique().tolist()
test_songs = test_df.iloc[:,1].unique().tolist()
test_couples = list(zip(test_df.iloc[:,0], test_df.iloc[:,1]))

all_users = list(set(train_users + test_users))
all_songs = list(set(train_songs + test_songs))
all_couples = list(itertools.product(all_users, all_songs))
new_couples = list(set(all_couples) - set(train_couples) - set(test_couples))
# new_couples = random.sample(new_couples, k=int(len(new_couples) * 0.01)) # taking a fraction of zeros



# Part 1 - Least Squares

In [130]:
n = df.shape[0]
m = n + len(new_couples)
avg = df.iloc[:,2].mean()*(n/m)
avg

1.8880274258705607

In [131]:
num_users = len(all_users)  # Number of users
num_songs = len(all_songs)  # Number of songs
num_songs, num_users

(16548, 1892)

In [132]:
# user dictionary
users_hash = {}
for i,user in enumerate(all_users):
    users_hash[user] = i

In [133]:
# song dictionary
songs_hash = {}
for j,song in enumerate(all_songs):
    songs_hash[song] = j + num_users

## create the mat for LS

In [134]:
train_users,train_songs =df['user_id'].map(users_hash).values, df['song_id'].map(songs_hash).values
merged_train = np.column_stack((train_users,train_songs)).reshape(-1)

In [135]:
rest_users, rest_items = zip(*new_couples)
users_list=[users_hash[user] for user in rest_users]
items_list=[songs_hash[song] for song in rest_items]
merged_rest = np.column_stack((users_list,items_list)).reshape(-1)

In [136]:
A_cols = np.concatenate((merged_train,merged_rest))

In [137]:
rows1,rows2 = np.arange(int(0.5*len(A_cols))),np.arange(int(0.5*len(A_cols)))
A_rows = np.column_stack((rows1,rows2)).reshape(-1)

In [138]:
values = np.ones(len(A_rows))

In [139]:
len(A_cols),len(A_rows),len(values)

(62611832, 62611832, 62611832)

In [140]:
from scipy.sparse import coo_matrix,csr_matrix,csc_matrix

A = coo_matrix((values, (A_rows, A_cols)),\
                    shape=(int(0.5*len(A_cols)),len(all_users)+len(all_songs))).tocsr()


## create c for LS

In [141]:
c_train = df['weight'].values - avg
c_rest = np.full(len(new_couples),-avg)
c = np.concatenate((c_train,c_rest))
# type(b_train), type(b_rest)

## Ls

In [142]:
from scipy.sparse.linalg import lsqr
final_b = lsqr(A=A,b=c)
final_b = final_b[0]

## predictions - part 1

In [172]:
b_u_hat = final_b[:num_users]
b_i_hat = final_b[num_users:]
user_biases_mat = np.repeat(b_u_hat, num_songs).reshape(
    (num_users, num_songs))  # duplicate to rows
item_biases_mat = np.tile(b_i_hat, num_users).reshape(
    (num_users, num_songs))
avg_mat = np.full((num_users, num_songs), avg)
predictions = avg_mat + user_biases_mat + item_biases_mat
predictions_part1 = np.maximum(predictions,0)

## true matrix

In [144]:
songs_hash = {}
for j, song in enumerate(all_songs):
    songs_hash[song] = j

In [145]:
values = df['weight'].values
rows = df['user_id'].map(users_hash).values
cols = df['song_id'].map(songs_hash).values
true_mat = coo_matrix((values, (rows, cols)), shape=predictions.shape).tocsr()


In [146]:
test_users_mapped = test_df['user_id'].map(users_hash).values
test_songs_mapped = test_df['song_id'].map(songs_hash).values
test_indxs_mapped = list((zip(test_users_mapped,test_songs_mapped)))

In [147]:
score_mat = true_mat-predictions
for idx in test_indxs_mapped:
    score_mat[idx] = 0

## Train scores - part 1

In [174]:
def get_score(true,pred,N):
    score_matrix =true-pred
    for indx in test_indxs_mapped:
        score_mat[indx] = 0
    return {'SSE': la.norm(score_matrix)**2, 'RMSE':((la.norm(score_matrix) ** 2) / N) ** 0.5}

In [175]:
N = num_users*num_songs - len(test_indxs_mapped)
scores_part1 = get_score(true_mat, predictions_part1, N)
print(f'RMSE in Part 1 on train set: {scores_part1["RMSE"]}\n'
      f'SSE in Part 1 on train set: {scores_part1["SSE"]}')

RMSE in Part 1 on train set: 187.47494295861532
SSE in Part 1 on train set: 1100300951732.863


## Test Prediction - part 1

In [ ]:
test_pred_1 = []
for i, row in test_df.iterrows():
    user = row['user_id']
    song = row['song_id']

    user_index = users_hash[user]
    bu = b_u_hat[user_index][0]

    song_index = songs_hash[song]
    bi = b_i_hat[song_index-num_users][0]
    test_pred_1.append(avg + bu + bi)
test_pred_1 = np.maximum(test_pred_1,0)

## part 1 - predicated!!

## create the LS matrix

In [9]:
num_records = df.shape[0]
A = sp.lil_matrix((num_records, num_users + num_songs))
c = np.full(num_records, -r_avg)

for i, row in df.iterrows():
    # Set the nonzero entries in the current row
    user = users_hash[row['user_id']]
    song = songs_hash[row['song_id']]
    A[i, user] = 1
    A[i, song] = 1
    c[i] += row['weight']

A = A.tocsc()
c = c.reshape((len(c), 1))

## perform gradient descent for LS

In [11]:
def step_size(AXC):
    step_numerator = (la.norm(A.transpose().dot(AXC))) ** 3
    step_denominator = (la.norm(A.dot(A.transpose().dot(AXC)))) ** 2
    return step_numerator / step_denominator


def grad(AXC):

    grad_numerator = A.transpose().dot(AXC)
    grad_denominator = la.norm(grad_numerator)
    return grad_numerator / grad_denominator


# gradient descent least squares:
max_iterations = 10**6
iteration = 0
epsilon = 0.001
gradient = 2 * epsilon
x0 = np.random.random((A.shape[1], 1))
x_new = x0
f_vals = [np.inf]
first = True
while first or (abs(la.norm(f_vals[-1])-la.norm(f_vals[-2])) >= epsilon and iteration<=max_iterations):
    first = False
    x_prev = x_new
    AX = A.dot(x_prev)
    AXC = AX - c
    f_vals.append(la.norm(AXC))
    gradient = grad(AXC)
    # print(la.norm(AXC))
    step = step_size(AXC)
    x_new = x_prev - (step * gradient)
    iteration += 1
    # print(iteration)


1048561.1958076822
1
1024865.6997130646
2
1020372.3372163313
3
1018647.4560300034
4
1017696.1308030526
5
1017088.8805048653
6
1016673.0226228106
7
1016375.899287761
8
1016157.3831916937
9
1015993.2366633832
10
1015867.8703221263
11
1015770.8184313382
12
1015694.8143090488
13
1015634.6930668377
14
1015586.7069594894
15
1015548.0964226071
16
1015516.79964296
17
1015491.2599618526
18
1015470.2885258462
19
1015452.9698801278
20
1015438.5922028866
21
1015426.5981629011
22
1015416.5476313138
23
1015408.0909296651
24
1015400.948098525
25
1015394.8937984004
26
1015389.7453976574
27
1015385.3541804826
28
1015381.5982996996
29
1015378.3775109202
30
1015375.6088901877
31
1015373.223596674
32
1015371.164204503
33
1015369.3826635092
34
1015367.8385974952
35
1015366.4979903366
36
1015365.3320768808
37
1015364.3164786713
38
1015363.4304666503
39
1015362.6563818681
40
1015361.9791365175
41
1015361.3858190784
42
1015360.8653512181
43
1015360.4082146139
44
1015360.0062116743
45
1015359.6522740513
46
101

# Part 2 - ALS Method

In [19]:
# user dictionary
users_hash = {}
for i,user in enumerate(all_users):
    users_hash[user] = i

In [20]:
# song dictionary
songs_hash = {}
for j,song in enumerate(all_songs):
    songs_hash[song] = j

In [21]:
num_users = len(all_users)  # Number of users
num_songs = len(all_songs)  # Number of songs
num_users, num_songs

(1892, 16548)

## create the ratings matrix

In [166]:
mat_part2 = true_mat.tolil(copy=True).astype(float)

In [167]:
for idx in test_indxs_mapped:
    mat_part2[idx] = np.nan

In [168]:
mat_part2_rows =mat_part2.tocsr(copy=True)
mat_part2_cols= mat_part2.tocsc(copy=True)

In [196]:
print(mat_part2_rows.getrow(0).toarray().flatten().shape)

(16548,)


In [213]:
# alternating least squares step
def als_step(latent_vectors, fixed_vecs, ratings, _lambda=0, type='user'):
    """
    One of the two ALS steps. Solve for the latent vectors
    specified by type.
    """
    if type == 'user':
        # calculating P, Q is fixed

        QTQ = fixed_vecs.T.dot(fixed_vecs)
        lambdaI = np.eye(QTQ.shape[0]) * _lambda

        for u in range(latent_vectors.shape[0]):  # iterate over P rows and R Rows
            r = ratings.getrow(u).toarray().flatten() # deep copy?
            rows_to_remove = np.where(np.isnan(r))[0]
            clean_r = r[~np.isnan(r)]
            mat = np.delete(fixed_vecs,rows_to_remove,axis=0)
            new_vec = la.lstsq(mat,clean_r,rcond=None)[0]
            # if u==0:
                # print(f"prev vec user {u}: {latent_vectors[u,:]}")
            # new_vec = (la.solve((QTQ + lambdaI),
            #                                  (ratings[u, :].dot(fixed_vecs)).transpose())).ravel()
            # if u==0:
                # print(new_vec)
            latent_vectors[u, :] = new_vec.ravel()
                # print(f"{np.array_equal(latent_vectors[u,:],new_vec)}")
    elif type == 'item':
        # calculating Q, P is fixed
        PTP = fixed_vecs.T.dot(fixed_vecs)
        lambdaI = np.eye(PTP.shape[0]) * _lambda

        for i in range(latent_vectors.shape[0]):
            # remove rows from
            r = ratings.getcol(i).toarray()
            rows_to_remove = np.where(np.isnan(r))[0]
            clean_r = r[~np.isnan(r)]
            mat = np.delete(fixed_vecs,rows_to_remove,axis=0)
            new_vec = la.lstsq(mat,clean_r,rcond=None)[0]
            # if i ==0:
            #     print(f"prev vec song {i}: {latent_vectors[i,:]}")
            # new_vec =  (la.solve((PTP + lambdaI),
            #                                  (ratings[:, i].transpose().dot(fixed_vecs)).transpose())).ravel()
            # # if i==0:
                # print(new_vec)
            latent_vectors[i, :] = new_vec.ravel()
                # print(f"{np.array_equal(latent_vectors[i,:],new_vec)}")
    return latent_vectors

In [222]:
def als_predict_vecs(checkpoints, iterations = 10, k = 20,learning_curve=False):
    np.random.seed(0)
    new_users = np.random.random((num_users, k))  # TODO: change to ones, but it throws an error
    new_songs = np.zeros((num_songs, k))
    scores_arr = []
    for k in range(iterations): #or not (np.array_equal(old_songs, new_songs) and np.array_equal(old_users,
        # new_users)):
        new_songs = als_step(new_songs,
                               new_users,
                               mat_part2_cols,
                               type='item')
        new_users = als_step(new_users,
                               new_songs,
                               mat_part2_rows,
                               type='user')
        if learning_curve:
            if k in checkpoints:
                meantime_pred = predict(new_users,new_songs)
                meantime_pred = np.maximum(meantime_pred,0)
                scores_arr.append(get_score(true_mat,meantime_pred,N))
    return new_users, new_songs, scores_arr


## Train Prediction - part 2

In [178]:
# user_matrix,song_matrix = als_predict()
def predict(users, songs):
    return np.matmul(users, songs.T)

In [223]:
def run_part_2(iterations):
    user_vecs, song_vecs, scores_arr = als_predict_vecs(checkpoints=[2,5,10,15,19],iterations=iterations,
                                                        learning_curve=True)
    print(f'{iterations} iterations')
    return scores_arr
    # predictions_p2 = predict(user_vecs, song_vecs)
    # predictions_p2 = np.maximum(predictions_p2,0)
    # return get_score(true_mat,predictions_p2,N),user_vecs, song_vecs

## check the learning curve

In [224]:
K=20
scores_part2,user_matrix,song_matrix = run_part_2(iterations=20) # TODO: 10?, 2 works better though
print(f'RMSE in Part 2 on train set: {scores_part2["RMSE"]}\n'
      f'SSE in Part 2 on train set: {scores_part2["SSE"]}\n'
      f'compared to part 1\n'
      f'Improvement of {scores_part1["RMSE"] - scores_part2["RMSE"]} in the RMSE\n'
      f'Improvement of {scores_part1["SSE"] - scores_part2["SSE"]} in the SSE')

KeyboardInterrupt: 

In [225]:
scores_test = run_part_2(iterations=20)

20 iterations


In [ ]:
RMSE_curve = [d['RMSE'] for d in scores_test]
SSE_curve = [d['SSE'] for d in scores_test]

## Test Prediction

In [ ]:
mean_user_entry = user_matrix.mean()
median_user_entry = np.median(user_matrix)

mean_song_entry = song_matrix.mean()
median_song_entry = np.median(song_matrix)

In [144]:
test_pred_2 = []
for i, row in test_df.iterrows():
    user = row['user_id']
    song = row['song_id']
    user_index = users_hash[user]
    user_vec = user_matrix[user_index]
    song_index = songs_hash[song]
    song_vec = song_matrix[song_index]
    test_pred_2.append(max(user_vec.dot(song_vec),0))


## predicated

In [23]:

# create the sparse ratings matrix
no_zeros = pd.read_csv('user_song.csv')
R = sp.lil_matrix((num_users, num_songs))

for i, row in no_zeros.iterrows():
    # Set the nonzero entries in the current row
    user = users_hash[row['user_id']]
    song = songs_hash[row['song_id']]
    rating = row['weight']
    R[user, song] = rating

R_rows = R.tocsr()  # we'll use to slice rows
R_cols = R.tocsc()  # we'll use to slice column

# Part 3

In [37]:
# Perform SVD on the sparse matrix
# k = Number of singular values/vectors to use
U, S, V = svds(R_rows, k=20)

# Reconstruct the matrix using k singular values/vectors
reduced_R = U.dot(sp.diags(S).dot(V))



494270.1826472662

## part 3 train SSE

In [38]:
la.norm(R - reduced_R)**2

244303013454.1619

part 3 test

In [41]:
test_pred_3 = []
for i, row in test_df.iterrows():
    user = row['user_id']
    song = row['song_id']
    user_index = users_hash[user]
    song_index = songs_hash[song]
    test_pred_3.append(max(reduced_R[user_index,song_index],0))

# ??

In [173]:
from sklearn.decomposition import TruncatedSVD
movie_svd = TruncatedSVD(n_components=20)
movie_features = movie_svd.fit_transform(R_rows)
movie_features


array([[ 1.61986513e+02,  8.16646444e-01,  4.15984691e+01, ...,
         1.74729460e+01, -2.21394839e+01,  1.45697821e+01],
       [ 9.83777088e-01,  1.04461350e-01,  2.58215767e-01, ...,
        -2.58216215e-01,  7.63471391e-01,  1.75510303e+00],
       [ 1.72788891e+03,  2.87772251e+00,  5.79312401e+01, ...,
        -5.16097506e+00,  1.75152615e+02, -2.76223694e+01],
       ...,
       [ 4.91562118e+00,  1.20800706e+01,  2.17293498e+03, ...,
        -1.99362494e+02, -4.39169951e+02,  1.83718364e+03],
       [ 2.32986067e+00,  1.15089461e+01,  5.04095525e+02, ...,
         2.63574394e+01,  2.70538707e+02, -5.24158997e+01],
       [ 5.74155471e+00,  3.02752030e+01,  1.36511058e+03, ...,
        -4.46342309e+01,  2.22556927e+02,  2.02332855e+02]])

In [3]:
import numpy as np

vector = np.array([1, 2, 3])
n = vector.shape[0]
m = 4 # Desired number of columns in the resulting matrix

matrix = vector.reshape(n, 1) * np.ones((1, m))

print(matrix)


[[1. 1. 1. 1.]
 [2. 2. 2. 2.]
 [3. 3. 3. 3.]]


In [53]:
np.zeros(3).shape[0]

3